# Get binary files (and transfrom them as .obj so we can later see them in Blender)

In [ ]:
import os

SIZE = 256

# Get outline voxels with N6
def is_surface(lines, line_idx, x):

    if lines[line_idx][x] != "1":
        return False

    z = line_idx // SIZE
    y = line_idx % SIZE

    neighbors = [
        (1,0,0), (-1,0,0),
        (0,1,0), (0,-1,0),
        (0,0,1), (0,0,-1)
    ]

    for dx, dy, dz in neighbors:

        nx = x + dx
        ny = y + dy
        nz = z + dz

        if nx < 0 or ny < 0 or nz < 0:
            return True
        if nx >= SIZE or ny >= SIZE or nz >= SIZE:
            return True

        n_line = nz*SIZE + ny

        if lines[n_line][nx] == "0":
            return True
    return False


# Create binary txt with outline
def create_surface_txt(input_path, output_path):

    with open(input_path, "r") as f:
        lines = [line.strip() for line in f]

    new_lines = []

    for line_idx in range(len(lines)):

        row = lines[line_idx]
        new_row = ""

        for x in range(SIZE):

            if is_surface(lines, line_idx, x):
                new_row += "1"
            else:
                new_row += "0"

        new_lines.append(new_row)

    with open(output_path, "w") as f:
        for r in new_lines:
            f.write(r + "\n")

    return new_lines

# Export .obj (with points)
def export_obj(surface_lines, obj_path):

    with open(obj_path, "w") as f:

        for line_idx in range(len(surface_lines)):

            row = surface_lines[line_idx]

            z = line_idx // SIZE
            y = line_idx % SIZE

            for x in range(SIZE):

                if row[x] == "1":
                    f.write(f"v {x} {y} {z}\n")


# Main
folder = "./3d_obj"

for file in os.listdir(folder):

    if file.endswith(".txt"):

        print("Currently at:", file)

        input_file = os.path.join(folder, file)

        surface_txt = os.path.join(folder, "outline_" + file)
        surface_obj = os.path.join(
            folder,
            "outline_" + file.replace(".txt", ".obj")
        )

        surface_lines = create_surface_txt(input_file, surface_txt)
        export_obj(surface_lines, surface_obj)

        print("Created:", surface_txt)
        print("Created:", surface_obj)
        print("\n")

Currently at: brain.txt
Created: ./3d_obj\outline_brain.txt
Created: ./3d_obj\outline_brain.obj


Currently at: bunny.txt
Created: ./3d_obj\outline_bunny.txt
Created: ./3d_obj\outline_bunny.obj


Currently at: cow.txt
Created: ./3d_obj\outline_cow.txt
Created: ./3d_obj\outline_cow.obj


Currently at: golfball.txt
Created: ./3d_obj\outline_golfball.txt
Created: ./3d_obj\outline_golfball.obj


Currently at: horse.txt
Created: ./3d_obj\outline_horse.txt
Created: ./3d_obj\outline_horse.obj


Currently at: igea.txt
Created: ./3d_obj\outline_igea.txt
Created: ./3d_obj\outline_igea.obj


Currently at: lion.txt
Created: ./3d_obj\outline_lion.txt
Created: ./3d_obj\outline_lion.obj


Currently at: lucy.txt
Created: ./3d_obj\outline_lucy.txt
Created: ./3d_obj\outline_lucy.obj


Currently at: pear.txt
Created: ./3d_obj\outline_pear.txt
Created: ./3d_obj\outline_pear.obj


Currently at: torus.txt
Created: ./3d_obj\outline_torus.txt
Created: ./3d_obj\outline_torus.obj




## Get .obj files as mesh (to work with blender)

In [ ]:
import os

SIZE = 256

def load_lines(path):
    with open(path, "r") as f:
        return [line.strip() for line in f]


def get_voxel(lines, x, y, z):
    if x < 0 or y < 0 or z < 0:
        return 0
    if x >= SIZE or y >= SIZE or z >= SIZE:
        return 0

    line_idx = z*SIZE + y
    return 1 if lines[line_idx][x] == "1" else 0


def add_face(vertices, faces, vdict, quad):

    idxs = []

    for v in quad:
        if v not in vdict:
            vdict[v] = len(vertices) + 1
            vertices.append(v)
        idxs.append(vdict[v])

    # Two triangles
    faces.append((idxs[0], idxs[1], idxs[2]))
    faces.append((idxs[0], idxs[2], idxs[3]))


def voxel_to_mesh(lines):

    vertices = []
    faces = []
    vdict = {}

    directions = [
        (1,0,0),(-1,0,0),
        (0,1,0),(0,-1,0),
        (0,0,1),(0,0,-1)
    ]

    for line_idx in range(len(lines)):

        z = line_idx // SIZE
        y = line_idx % SIZE

        for x in range(SIZE):

            if lines[line_idx][x] != "1":
                continue

            for dx,dy,dz in directions:

                if get_voxel(lines,x+dx,y+dy,z+dz)==1:
                    continue

                # Exposed face, then create quad
                if (dx,dy,dz)==(1,0,0):
                    quad=[(x+1,y,z),(x+1,y+1,z),(x+1,y+1,z+1),(x+1,y,z+1)]

                elif (dx,dy,dz)==(-1,0,0):
                    quad=[(x,y,z),(x,y,z+1),(x,y+1,z+1),(x,y+1,z)]

                elif (dx,dy,dz)==(0,1,0):
                    quad=[(x,y+1,z),(x,y+1,z+1),(x+1,y+1,z+1),(x+1,y+1,z)]

                elif (dx,dy,dz)==(0,-1,0):
                    quad=[(x,y,z),(x+1,y,z),(x+1,y,z+1),(x,y,z+1)]

                elif (dx,dy,dz)==(0,0,1):
                    quad=[(x,y,z+1),(x+1,y,z+1),(x+1,y+1,z+1),(x,y+1,z+1)]

                elif (dx,dy,dz)==(0,0,-1):
                    quad=[(x,y,z),(x,y+1,z),(x+1,y+1,z),(x+1,y,z)]

                add_face(vertices,faces,vdict,quad)

    return vertices,faces


def write_obj(path,vertices,faces):

    with open(path,"w") as f:

        for v in vertices:
            f.write(f"v {v[0]} {v[1]} {v[2]}\n")

        for tri in faces:
            f.write(f"f {tri[0]} {tri[1]} {tri[2]}\n")


# Main
folder="./3d_obj"

for file in os.listdir(folder):

    if file.endswith(".txt"):

        print("Meshing ",file)

        lines=load_lines(os.path.join(folder,file))

        verts,faces=voxel_to_mesh(lines)

        out=os.path.join(folder,file.replace(".txt","_mesh.obj"))

        write_obj(out,verts,faces)

        print("Vertices:",len(verts))
        print("Faces:",len(faces))
        print("Saved:",out)

Meshing  brain.txt
Vertices: 346474
Faces: 694200
Saved: ./3d_obj\brain_mesh.obj
Meshing  bunny.txt
Vertices: 232028
Faces: 464444
Saved: ./3d_obj\bunny_mesh.obj
Meshing  cow.txt
Vertices: 91266
Faces: 182576
Saved: ./3d_obj\cow_mesh.obj
Meshing  golfball.txt
Vertices: 312366
Faces: 624728
Saved: ./3d_obj\golfball_mesh.obj
Meshing  horse.txt
Vertices: 100956
Faces: 201936
Saved: ./3d_obj\horse_mesh.obj
Meshing  igea.txt
Vertices: 229016
Faces: 458032
Saved: ./3d_obj\igea_mesh.obj
Meshing  lion.txt
Vertices: 90860
Faces: 181816
Saved: ./3d_obj\lion_mesh.obj
Meshing  lucy.txt
Vertices: 93450
Faces: 187276
Saved: ./3d_obj\lucy_mesh.obj
Meshing  outline_brain.txt
Vertices: 550658
Faces: 1380320
Saved: ./3d_obj\outline_brain_mesh.obj
Meshing  outline_bunny.txt
Vertices: 363153
Faces: 910280
Saved: ./3d_obj\outline_bunny_mesh.obj
Meshing  outline_cow.txt
Vertices: 142827
Faces: 351880
Saved: ./3d_obj\outline_cow_mesh.obj
Meshing  outline_golfball.txt
Vertices: 482144
Faces: 1240792
Saved: ./